<a href="https://colab.research.google.com/github/htbkoo/0Colaboratory/blob/master/Kaggle/titanic-machine-learning-from-disaster/(xgboost)_htbkoo_Titanic__Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

## Limiting GPU memory growth

By default, TensorFlow maps nearly all of the GPU memory of all GPUs (subject to
[`CUDA_VISIBLE_DEVICES`](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars)) visible to the process. This is done to more efficiently use the relatively precious GPU memory resources on the devices by reducing memory fragmentation. To limit TensorFlow to a specific set of GPUs we use the `tf.config.experimental.set_visible_devices` method.

In [15]:
import tensorflow as tf

TF_MEMORY_GROWTH_CONFIG = True

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, TF_MEMORY_GROWTH_CONFIG)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")  # hiding to avoid leaking information into outputs
    print(f"ok, 'set_memory_growth' for all gpus to {TF_MEMORY_GROWTH_CONFIG}")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

ok, 'set_memory_growth' for all gpus to True


### Actual Notebook

#### Imports

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [25]:
import tensorflow as tf
import autokeras as ak

#### Train / Test data setup

In [17]:
data_folder = './local_data'
print(os.listdir(data_folder))

['test.csv', 'train.csv']


In [18]:
train_data_path = f'{data_folder}/train.csv'
test_data_path = f'{data_folder}/test.csv'

In [26]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True,
    max_trials=3) # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    # The path to the train.csv file.
    train_data_path,
    # The name of the label column.
    'Survived',
    epochs=10)
# Predict with the best model.
predicted_y = clf.predict(test_data_path)
# Evaluate the best model with testing data.
# print(clf.evaluate(test_file_path, 'survived'))


Trial 3 Complete [00h 00m 02s]
val_loss: 0.3246705234050751

Best val_loss So Far: 0.1598200649023056
Total elapsed time: 00h 00m 06s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
28/28 [==============================] - 0s 2ms/step - loss: 0.3800 - mean_squared_error: 0.3800
Epoch 2/10
28/28 [==============================] - 0s 2ms/step - loss: 0.2253 - mean_squared_error: 0.2253
Epoch 3/10
28/28 [==============================] - 0s 2ms/step - loss: 0.1807 - mean_squared_error: 0.1807
Epoch 4/10
28/28 [==============================] - 0s 2ms/step - loss: 0.1618 - mean_squared_error: 0.1618
Epoch 5/10
28/28 [==============================] - 0s 2ms/step - loss: 0.1509 - mean_squared_error: 0.1509
Epoch 6/10
28/28 [==============================] - 0s 2ms/step - loss: 0.1428 - mean_squared_error: 0.1428
Epoch 7/10
28/28 [==============================] - 0s 2ms/step - loss: 0.1368 - mean_squared_error: 0.1368
Epoch 8/10
28/28 [==============================] - 0s 2ms/step - loss: 

KeyError: 'Survived'

In [27]:
predicted_y

array([[ 1.77632302e-01],
       [ 4.52613115e-01],
       [ 3.82916987e-01],
       [ 2.49574363e-01],
       [ 4.08785194e-01],
       [ 2.64195949e-01],
       [ 2.98569173e-01],
       [ 3.06525379e-01],
       [ 8.00926030e-01],
       [ 3.28331590e-02],
       [ 2.31569529e-01],
       [ 3.04002851e-01],
       [ 1.03733134e+00],
       [ 2.50858307e-01],
       [ 1.01848197e+00],
       [ 9.65063989e-01],
       [ 4.99152660e-01],
       [ 2.35689700e-01],
       [ 5.75449109e-01],
       [ 6.03925705e-01],
       [ 7.65643567e-02],
       [ 1.77708864e-01],
       [ 1.09720159e+00],
       [ 4.24487665e-02],
       [ 4.05896962e-01],
       [-4.08735052e-02],
       [ 6.06920123e-01],
       [ 2.35114545e-01],
       [ 3.14378709e-01],
       [-3.40920687e-01],
       [ 2.39385247e-01],
       [ 2.71109074e-01],
       [ 5.24964392e-01],
       [ 6.98009372e-01],
       [ 4.24074918e-01],
       [ 2.37057954e-01],
       [ 8.37816060e-01],
       [ 7.97066867e-01],
       [ 2.5

#### Preivew of Train / Test data

In [ ]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [19]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [20]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
test_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [22]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [23]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [24]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


#### Other / reference / deprecated

In [11]:
columns = test_data.columns
columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [12]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [13]:
ALL_FEATURES = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
ALL_FEATURES

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [14]:
# features = [ 'Pclass', 'Name', 'Sex', 'Age', 'SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
features = [ 'Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [11]:
train_X = train_data[features]
train_X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [15]:
# [col for col in train_data.columns if train_data[col].isnull().any()]

len(train_data[train_data['Age'].isnull()])

# len(train_data)

177

In [25]:
[col for col in train_data.columns if train_data[col].isnull().any()]

len(train_data[train_data['Age'].isnull()])

len(train_data)

891

In [ ]:
missing_val_count_by_column = (train_X.isnull().sum())
missing_val_count_by_column
# print(missing_val_count_by_column[missing_val_count_by_column > 0

In [ ]:
train_X["Sex"] =  [0 if ((sex == 'female') or (sex == 0)) else 1 for sex in train_X["Sex"]]
train_X["Embarked"] =  [0 if ((embarked == 'C') or (embarked == 0)) else 1 if ((embarked == 'Q') or (embarked == 1)) else 2 for embarked in train_X["Embarked"]]
train_X.head()

In [ ]:
age_mean = train_X.Age.mean()
age_mean

In [ ]:
train_X = train_X.fillna({'Age': age_mean, 'Embarked': 0})

In [ ]:
missing_val_count_by_column = (train_X.isnull().sum())
missing_val_count_by_column

In [ ]:
train_y = train_data.Survived
train_y.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state=1)

In [ ]:
rf_model.fit(train_X, train_y)

In [ ]:
test_X = test_data[features]
test_X.head()

In [ ]:
missing_val_count_by_column = (test_X.isnull().sum())
missing_val_count_by_column

In [ ]:
test_X["Sex"] =  [0 if ((sex == 'female') or (sex == 0)) else 1 for sex in test_X["Sex"]]
test_X["Embarked"] =  [0 if ((embarked == 'C') or (embarked == 0)) else 1 if ((embarked == 'Q') or (embarked == 1)) else 2 for embarked in test_X["Embarked"]]
test_X.head()

In [ ]:
test_age_mean = test_X.Age.mean()
test_fare_mean = test_X.Fare.mean()

In [ ]:
test_X = test_X.fillna({'Age': test_age_mean, 'Fare': test_fare_mean})

In [ ]:
missing_val_count_by_column = (test_X.isnull().sum())
missing_val_count_by_column

In [ ]:
test_preds = rf_model.predict(test_X)
test_preds[:10]

In [ ]:
threshold = 0.5
test_preds = [0 if pred<threshold else 1 for pred in test_preds]
test_preds[:10]

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId,
                      'Survived': test_preds})
output.to_csv('submission.csv', index=False)

In [ ]:
[col for col in [1,2,3,4] if col!=1]

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# read in data
dtrain = xgb.DMatrix(trains_X.values, trains_Y.values)
dtest = xgb.DMatrix(test_X.values)
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

mean_absolute_error(preds, )